In [1]:
# import os
# import sys

# spark_home=os.environ.get("SPARK_HOME",None)
# if not spark_home:
#     raise ValueError("SPARK_HOME environment variable is not set")
# sys.path.insert(0,os.path.join(spark_home,'python'))
# sys.path.insert(0,os.path.join(spark_home,'C:\spark\python\lib\py4j-0.10.7-src.zip'))
# fname=os.path.join(spark_home,"python\pyspark\shell.py")
# exec(compile(open(fname,"rb").read(),fname,'exec'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2
      /_/

Using Python version 3.6.5 (default, Mar 29 2018 13:32:41)
SparkSession available as 'spark'.


In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import math
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.stat import Statistics
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from numpy import array
from pyspark.sql import Row
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import PCA
from pyspark.ml.feature import StringIndexer
from pyspark.ml.regression import LinearRegression
import time


In [ ]:
#while True:
df = sc.textFile("data.csv")
header = df.first()
data = df.filter(lambda x: x !=header)
print(data.take(2))
   # time.sleep(5)

['60,30,72,flat,yes,1,2019/3/20,8:45:48,236520253.677', '47,54,54,flat,yes,1,2019/3/20,10:15:37,827800088.263']
['60,30,72,flat,yes,1,2019/3/20,8:45:48,236520253.677', '47,54,54,flat,yes,1,2019/3/20,10:15:37,827800088.263']
['60,30,72,flat,yes,1,2019/3/20,8:45:48,236520253.677', '47,54,54,flat,yes,1,2019/3/20,10:15:37,827800088.263']
['60,30,72,flat,yes,1,2019/3/20,8:45:48,236520253.677', '47,54,54,flat,yes,1,2019/3/20,10:15:37,827800088.263']
['60,30,72,flat,yes,1,2019/3/20,8:45:48,236520253.677', '47,54,54,flat,yes,1,2019/3/20,10:15:37,827800088.263']
['60,30,72,flat,yes,1,2019/3/20,8:45:48,236520253.677', '47,54,54,flat,yes,1,2019/3/20,10:15:37,827800088.263']
['60,30,72,flat,yes,1,2019/3/20,8:45:48,236520253.677', '47,54,54,flat,yes,1,2019/3/20,10:15:37,827800088.263']
['60,30,72,flat,yes,1,2019/3/20,8:45:48,236520253.677', '47,54,54,flat,yes,1,2019/3/20,10:15:37,827800088.263']
['60,30,72,flat,yes,1,2019/3/20,8:45:48,236520253.677', '47,54,54,flat,yes,1,2019/3/20,10:15:37,82780008

In [20]:
header = df.first()
data = df.filter(lambda x: x !=header)

In [21]:
data.take(2)

['83,53,66,flat,no,1,2019/3/20,8:33:17,252994267.311',
 '40,46,62,villa,yes,1,2019/3/20,8:33:22,330497139.995']

In [22]:
def converttonumber(inputstr):
    attlist = inputstr.split(",")
    home=0
    consum=0
    if(attlist[3]=='flat'):
        home=0
    else:
        home=1
    
    if(attlist[4]=='no'):
        consum=0
    else:
        consum=1
    values = Vectors.dense([float(attlist[0]),float(attlist[1]),float(attlist[2]),float(attlist[5]),home,consum,float(attlist[8])])
    return values
    

In [23]:
dfVectors= data.flatMap(converttonumber)
type(dfVectors)

pyspark.rdd.PipelinedRDD

In [24]:
sqlcontext=SQLContext(sc)

In [25]:
def transformToLabeledPoint(instr):
    attlist=instr.split(",")
    home=0
    consum=0
    if(attlist[3]=='flat'):
        home=0
    else:
        home=1
    
    if(attlist[4]=='no'):
        consum=0
    else:
        consum=1
    lp = (float(attlist[8]),Vectors.dense(float(attlist[0]),float(attlist[1]),float(attlist[2]),float(attlist[5]),home,consum))
    return lp

In [26]:
dfLP=data.map(transformToLabeledPoint)
dfLP.take(2)

[(252994267.311, DenseVector([83.0, 53.0, 66.0, 1.0, 0.0, 0.0])),
 (330497139.995, DenseVector([40.0, 46.0, 62.0, 1.0, 1.0, 1.0]))]

In [27]:
MyDf=sqlContext.createDataFrame(dfLP,["label","features"])
MyDf.select("label","features").show(2)

+---------------+--------------------+
|          label|            features|
+---------------+--------------------+
|2.52994267311E8|[83.0,53.0,66.0,1...|
|3.30497139995E8|[40.0,46.0,62.0,1...|
+---------------+--------------------+
only showing top 2 rows



In [28]:
dataPCA = PCA(k=3,inputCol="features",outputCol="pcaFeature")


In [29]:
(trainingData, testData) = MyDf.randomSplit([0.7, 0.3],42)
trainingData.count(),testData.count()

pyspark.sql.dataframe.DataFrame

In [30]:
lr = LinearRegression(maxIter=10)
lrModel = lr.fit(trainingData)

IllegalArgumentException: 'requirement failed: Column features must be of type org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7 but was actually org.apache.spark.mllib.linalg.VectorUDT@f71b0bce.'

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 13083)
Traceback (most recent call last):
  File "C:\Users\Mahmoud\Anaconda3\lib\socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\Mahmoud\Anaconda3\lib\socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\Mahmoud\Anaconda3\lib\socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\Mahmoud\Anaconda3\lib\socketserver.py", line 696, in __init__
    self.handle()
  File "c:\spark\python\pyspark\accumulators.py", line 265, in handle
    poll(accum_updates)
  File "c:\spark\python\pyspark\accumulators.py", line 238, in poll
    if func():
  File "c:\spark\python\pyspark\accumulators.py", line 242, in accum_updates
    num_updates = read_int(self.rfile)
  File "c:\spark\python\pyspark\seri